In [14]:
!pip install sentence-transformers

In [2]:
f = open('/content/drive/MyDrive/sample1.txt', 'r')
sample = f.read()
f.close()

In [3]:
print(sample)

All human beings are born free and equal in dignity and rights. They are endowed with reason and conscience and should act towards one another in a spirit of brotherhood.</br>
Everyone is entitled to all the rights and freedoms set forth in this Declaration, without distinction of any kind, such as race, colour, sex, language, religion, political or other opinion, national or social origin, property, birth or other status. Furthermore, no distinction shall be made on the basis of the political, jurisdictional or international status of the country or territory to which a person belongs, whether it be independent, trust, non-self-governing or under any other limitation of sovereignty.</br>
All are equal before the law and are entitled without any discrimination to equal protection of the law. All are entitled to equal protection against any discrimination in violation of this Declaration and against any incitement to such discrimination.


In [4]:
import re
from pprint import pprint
read = re.compile('\n| *</br> *\n? *')
sample_list = read.split(sample)
print(len(sample_list))

3


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import itertools

model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [37]:
def keyword_extraction_MSS(doc, n_gram_range, stop_words, top_n, nr_candidates):
  count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc]) #vectorizing n_gram candidates
  candidates = count.get_feature_names_out()
  
   #encoding part
  doc_embedding = model.encode([doc])
  candidate_embeddings = model.encode(candidates)

  distances = cosine_similarity(doc_embedding, candidate_embeddings)
  distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings) #caculating distances

  candidates_idx = list(distances.argsort()[0][-nr_candidates:])            # picking #nr_candidates candidates from distance
  candidates_val = [candidates[index] for index in candidates_idx]
  distances_candidates = distances_candidates[np.ix_(candidates_idx,candidates_idx)]   #picking cosine simil between candidates

  min_sim = np.inf

  candidate = None

  for combination in itertools.combinations(range(len(candidates_idx)), top_n):
    sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
    if sim < min_sim:
      min_sim = sim
      candidate = combination
  
  return [candidates_val[index] for index in candidate]

In [29]:
sample_list[0]

'All human beings are born free and equal in dignity and rights. They are endowed with reason and conscience and should act towards one another in a spirit of brotherhood.'

In [38]:
n_gram_range=(3,3)
stop_words = None
keywords=[]
top_n=5
nr_candidates=20
for i in range(len(sample_list)):
  temp = keyword_extraction_MSS(sample_list[i],n_gram_range, stop_words, top_n, nr_candidates)
  keywords.append(temp)



In [39]:
pprint(keywords)

[['and should act',
  'they are endowed',
  'reason and conscience',
  'all human beings',
  'are born free'],
 ['trust non self',
  'non self governing',
  'to all the',
  'rights and freedoms',
  'everyone is entitled'],
 ['law and are',
  'against any incitement',
  'discrimination to equal',
  'the law all',
  'without any discrimination']]
